In [1]:
import h5py
import os.path

def save_file(folder,name,data,xaxis,yaxis):
    path = folder + '/' + name
    if not os.path.exists(path):
        with h5py.File(path, 'w') as f:
            f.create_dataset('data', data = data)
            f.create_dataset('X_'+xaxis, data = 0)
            f.create_dataset('Y_'+yaxis, data = 0)
            f.close()
            print('The file has been saved')
    else:
        print('The file already exists')
    return None

def rewrite_file(folder,name,data,xaxis,yaxis):
    path = folder + '/' + name
    if os.path.exists(path):
        with h5py.File(path, 'w') as f:
            f.create_dataset('data', data = data)
            f.create_dataset('X_'+xaxis, data = 0)
            f.create_dataset('Y_'+yaxis, data = 0)
            f.close()
            print('The file has been rewrited')
    else:
        print('The file does not exist')
    return None

def read_file(folder,name):
    path = folder + '/' + name
    with h5py.File(path, 'r') as f:
        data = f['data'][:]
        print(f.keys())
        f.close()
    print('Data shape =',data.shape)
    return data

In [43]:
import numpy as np
import torch

In [49]:
A = torch.tensor(np.arange(10).reshape(2,5))

In [50]:
import datetime

datetime.datetime.now()

datetime.datetime(2022, 1, 7, 18, 51, 46, 253132)

In [62]:
path = 'Q'
with h5py.File(path, 'w') as f:
    f.create_dataset('data', data = A)
    f.create_dataset('data1', data = A)

In [63]:
with h5py.File(path, 'r') as f:
    data = f['data1'][:]
data

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [53]:
import torch
import torch.nn as nn

class function(torch.nn.Module):
    def __init__(self,th=None):
        super(function, self).__init__()
        self.th = th
        
    def forward(self, x):
        return x * self.th

    def extra_repr(self):
        return 'opt_th={}'.format(self.th)
    
system = nn.Sequential()
inp  = 2
n = 3
for i in range(0,n):
    # find optimal threshold
    th = system.forward(inp)
    # add this layer into model
    system.add_module('layer_'+str(i),function(th))
    
system.named_modules, system.forward(3)

(<bound method Module.named_modules of Sequential(
   (layer_0): function(opt_th=2)
   (layer_1): function(opt_th=4)
   (layer_2): function(opt_th=16)
 )>,
 384)

In [38]:
class Computer():
    def __init__(self, computer, ram, ssd):
        self.computer = computer
        self.ram = ram
        self.ssd = ssd

# Если создать дочерний класс `Laptop`, то будет доступ 
# к свойству базового класса благодаря функции super().
class Laptop(Computer):
    def __init__(self, computer, ram, ssd, model):
        super(Laptop).__init__()
        self.model = model


In [39]:
lap1 = Computer('lenovo',512,256)

In [40]:
lap1.computer

'lenovo'

In [48]:
lap2 = Laptop('lenovo',512,256,'ssw')

In [49]:
lap2.model

'ssw'

In [50]:
lap2.m